In [59]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timedelta
import pytz
import json
import ta_formulas as ta
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry
import boto3
import ast

s3 = boto3.client('s3')

In [60]:
def pull_closed_orders_s3(path, bucket):
    dfs = []
    keys = s3.list_objects(Bucket=bucket,Prefix=f"{path}")["Contents"]
    for object in keys:
        key = object['Key']
        dataset = s3.get_object(Bucket=bucket,Key=f"{key}")
        df = pd.read_csv(dataset.get("Body"))
        df.drop(columns=['Unnamed: 0'],inplace=True)
        if len(df) > 0:
            dfs.append(df)
    full_df = pd.concat(dfs)
    full_df.reset_index(drop=True)
    return full_df

In [61]:
path = 'closed_orders/PROD_VAL/CDBFC_1D'
bucket = 'inv-alerts-trading-data'

df = pull_closed_orders_s3(path, bucket)

df.head()

,open_order_id,closing_order_id,option_symbol,position_id
0,10831957,10832538,UBER240322C00079000,UBER-CDBFC1D-2024-03-18T15-05
1,10831962,10832541,UBER240322C00080000,UBER-CDBFC1D-2024-03-18T15-05
0,10831952,10833538,IWM240322C00209000,IWM-CDBFC1D-2024-03-18T15-05
0,10835525,10846786,SNOW240322C00170000,SNOW-CDBFC1D-2024-03-18T17-07
1,10833362,10846791,SNOW240322C00170000,SNOW-CDBFC1D-2024-03-18T16-06


In [62]:
orders_list = df['open_order_id'].to_list()

In [63]:
client = boto3.client('dynamodb')

def extract_values_from_dict(d):
    return list(d.values())[0] if isinstance(d, dict) else d

def get_open_trades_by_orderid(order_id_list):
    partitions = break_array_into_partitions(order_id_list)
    df_list = []
    error_list = []
    for partition in partitions:
        try:
            response = client.batch_get_item(
                RequestItems={
                    'icarus-closed-orders-table-inv': {
                        'Keys': [{'order_id': {'S': str(id)}} for id in partition]
                    }
                }
            )
            
            items = response['Responses']['icarus-closed-orders-table-inv']
            result_df = pd.DataFrame(items)
            result_df = result_df.map(extract_values_from_dict)
            df_list.append(result_df)
        except:
            error_list.append(partition)
    full_df = pd.concat(df_list)
    return full_df, error_list

def break_array_into_partitions(arr):
    """
    Breaks an array into equal partitions of less than 100 items each.

    Args:
        arr (list): The input array.

    Returns:
        list: A list of partitions, where each partition is a list of items.
    """
    if len(arr) <= 100:
        # If the array has 100 or fewer items, return it as-is
        return [arr]
    else:
        # Calculate the number of partitions needed
        num_partitions = (len(arr) + 99) // 100  # Round up to the nearest integer

        # Calculate the size of each partition
        partition_size = len(arr) // num_partitions

        # Initialize the starting index and ending index for each partition
        start = 0
        end = partition_size

        # Create the partitions
        partitions = []
        for i in range(num_partitions):
            # If it's the last partition, include any remaining items
            if i == num_partitions - 1:
                partitions.append(arr[start:])
            else:
                partitions.append(arr[start:end])

            # Update the starting and ending index for the next partition
            start = end
            end += partition_size

        return partitions

In [64]:
orders_df, errors = get_open_trades_by_orderid(orders_list)

In [65]:
orders_df.head(10)

,last_fill_price_close,close_creation_date,qty_executed_close,last_fill_price_open,qty_executed_open,order_transaction_date,env,position_id,order_creation_date,average_fill_price_close,closing_order_id,close_transaction_date,order_id,option_symbol,underlying_symbol,average_fill_price_open,execution_strategy,close_reason,spread_position
0,0.31,2024-03-18T14:07:00.228Z,2.0,0.13,2.0,2024-03-18T14:07:00.374Z,PROD_VAL,C-CDBFC1D-2024-03-18T14-06,2024-03-18T14:07:00.228Z,0.31,10846855,2024-03-19T13:50:40.453Z,10829666,C240322C00060000,C,0.13,None,"Hit exit target, sell.",NaN
1,0.01,2024-03-19T18:06:31.533Z,1.0,0.02,1.0,2024-03-19T18:06:31.564Z,PROD_VAL,BAC-CDBFC1D-2024-03-19T18-06,2024-03-19T18:06:31.533Z,0.01,10866346,2024-03-20T13:50:59.765Z,10854459,BAC240322C00038000,BAC,0.02,None,"Failed momentum gate, sell.",NaN
2,0.01,2024-03-19T16:06:36.962Z,1.0,0.03,1.0,2024-03-19T16:06:37.130Z,PROD_VAL,CVX-CDBFC1D-2024-03-19T16-06,2024-03-19T16:06:36.962Z,0.01,10866220,2024-03-20T13:50:36.799Z,10851392,CVX240322C00165000,CVX,0.03,None,"Hit point of no confidence, sell.",NaN
3,0.21,2024-03-19T17:06:43.586Z,1.0,0.22,1.0,2024-03-19T17:06:43.608Z,PROD_VAL,PLTR-CDBFC1D-2024-03-19T17-06,2024-03-19T17:06:43.586Z,0.21,10866241,2024-03-20T13:50:40.974Z,10853236,PLTR240322C00024500,PLTR,0.22,None,"Failed momentum gate, sell.",NaN
4,0.12,2024-03-18T18:06:27.510Z,1.0,0.31,1.0,2024-03-18T18:06:27.535Z,PROD_VAL,UBER-CDBFC1D-2024-03-18T18-06,2024-03-18T18:06:27.510Z,0.12,10846875,2024-03-19T13:50:45.210Z,10836841,UBER240322C00079000,UBER,0.31,None,"Hit point of no confidence, sell.",NaN
5,0.04,2024-03-18T17:07:13.350Z,1.0,0.09,1.0,2024-03-18T17:07:13.365Z,PROD_VAL,AAPL-CDBFC1D-2024-03-18T17-07,2024-03-18T17:07:13.350Z,0.04,10846947,2024-03-19T13:50:55.024Z,10835522,AAPL240322C00185000,AAPL,0.09,None,"Hit point of no confidence, sell.",NaN
6,0.05,2024-03-18T19:06:10.015Z,1.0,0.11,1.0,2024-03-18T19:06:10.039Z,PROD_VAL,PYPL-CDBFC1D-2024-03-18T19-06,2024-03-18T19:06:10.015Z,0.05,10846796,2024-03-19T13:50:32.306Z,10838605,PYPL240322C00069000,PYPL,0.11,None,"Hit point of no confidence, sell.",NaN
7,0.33,2024-03-18T16:06:13.797Z,1.0,0.32,1.0,2024-03-18T16:06:13.821Z,PROD_VAL,UBER-CDBFC1D-2024-03-18T16-06,2024-03-18T16:06:13.797Z,0.33,10846974,2024-03-19T13:50:58.572Z,10833360,UBER240322C00078000,UBER,0.32,None,"Failed momentum gate, sell.",NaN
8,0.13,2024-03-19T16:06:38.162Z,1.0,0.25,1.0,2024-03-19T16:06:38.194Z,PROD_VAL,C-CDBFC1D-2024-03-19T16-06,2024-03-19T16:06:38.162Z,0.13,10866267,2024-03-20T13:50:45.644Z,10851397,C240322C00060000,C,0.25,None,"Failed momentum gate, sell.",NaN
9,0.15,2024-03-19T14:06:36.715Z,1.0,0.31,1.0,2024-03-19T14:06:36.854Z,PROD_VAL,XOM-CDBFC1D-2024-03-19T14-06,2024-03-19T14:06:36.715Z,0.15,10866179,2024-03-20T13:50:30.876Z,10847797,XOM240322C00115000,XOM,0.31,None,"Failed momentum gate, sell.",NaN


In [71]:
orders_df['average_fill_price_close_float'] = orders_df['average_fill_price_close'].astype(float)
orders_df['average_fill_price_open_float'] = orders_df['average_fill_price_open'].astype(float)

In [77]:
def clean_orders_df(orders):
    #Need to add a column that indicates if it is a winner or a loser
    #Need to append rows with winners to a specific column and losers to another column
    #Separate out trades that hit the floor value
    #Find if there was a point at which the trade couldve been profitable
    #Create a new dataframe of all those trades and export as a CSV - could do one for each strategy, starting at CDBFC_1D
    # orders['pnlcoefficient'] = orders.apply(lambda row: row['average_fill_price_close_float'] - row['average_fill_price_open_float'])

    orders['pnlcoefficient'] = orders['average_fill_price_close_float'] - orders['average_fill_price_open_float']
    orders['winloss'] = np.where(orders['pnlcoefficient'] < 0, 'L', 'W')
    return orders



In [75]:
orders = clean_orders_df(orders_df)

,last_fill_price_close,close_creation_date,qty_executed_close,last_fill_price_open,qty_executed_open,order_transaction_date,env,position_id,order_creation_date,average_fill_price_close,...,option_symbol,underlying_symbol,average_fill_price_open,execution_strategy,close_reason,spread_position,average_fill_price_close_float,average_fill_price_open_float,pnlcoefficient,winloss
0,0.31,2024-03-18T14:07:00.228Z,2.0,0.13,2.0,2024-03-18T14:07:00.374Z,PROD_VAL,C-CDBFC1D-2024-03-18T14-06,2024-03-18T14:07:00.228Z,0.31,...,C240322C00060000,C,0.13,None,"Hit exit target, sell.",NaN,0.31,0.13,0.18,W
1,0.01,2024-03-19T18:06:31.533Z,1.0,0.02,1.0,2024-03-19T18:06:31.564Z,PROD_VAL,BAC-CDBFC1D-2024-03-19T18-06,2024-03-19T18:06:31.533Z,0.01,...,BAC240322C00038000,BAC,0.02,None,"Failed momentum gate, sell.",NaN,0.01,0.02,-0.01,L
2,0.01,2024-03-19T16:06:36.962Z,1.0,0.03,1.0,2024-03-19T16:06:37.130Z,PROD_VAL,CVX-CDBFC1D-2024-03-19T16-06,2024-03-19T16:06:36.962Z,0.01,...,CVX240322C00165000,CVX,0.03,None,"Hit point of no confidence, sell.",NaN,0.01,0.03,-0.02,L
3,0.21,2024-03-19T17:06:43.586Z,1.0,0.22,1.0,2024-03-19T17:06:43.608Z,PROD_VAL,PLTR-CDBFC1D-2024-03-19T17-06,2024-03-19T17:06:43.586Z,0.21,...,PLTR240322C00024500,PLTR,0.22,None,"Failed momentum gate, sell.",NaN,0.21,0.22,-0.01,L
4,0.12,2024-03-18T18:06:27.510Z,1.0,0.31,1.0,2024-03-18T18:06:27.535Z,PROD_VAL,UBER-CDBFC1D-2024-03-18T18-06,2024-03-18T18:06:27.510Z,0.12,...,UBER240322C00079000,UBER,0.31,None,"Hit point of no confidence, sell.",NaN,0.12,0.31,-0.19,L


In [76]:
orders.head(20)

,last_fill_price_close,close_creation_date,qty_executed_close,last_fill_price_open,qty_executed_open,order_transaction_date,env,position_id,order_creation_date,average_fill_price_close,...,option_symbol,underlying_symbol,average_fill_price_open,execution_strategy,close_reason,spread_position,average_fill_price_close_float,average_fill_price_open_float,pnlcoefficient,winloss
0,0.31,2024-03-18T14:07:00.228Z,2.0,0.13,2.0,2024-03-18T14:07:00.374Z,PROD_VAL,C-CDBFC1D-2024-03-18T14-06,2024-03-18T14:07:00.228Z,0.31,...,C240322C00060000,C,0.13,None,"Hit exit target, sell.",NaN,0.31,0.13,0.18,W
1,0.01,2024-03-19T18:06:31.533Z,1.0,0.02,1.0,2024-03-19T18:06:31.564Z,PROD_VAL,BAC-CDBFC1D-2024-03-19T18-06,2024-03-19T18:06:31.533Z,0.01,...,BAC240322C00038000,BAC,0.02,None,"Failed momentum gate, sell.",NaN,0.01,0.02,-0.01,L
2,0.01,2024-03-19T16:06:36.962Z,1.0,0.03,1.0,2024-03-19T16:06:37.130Z,PROD_VAL,CVX-CDBFC1D-2024-03-19T16-06,2024-03-19T16:06:36.962Z,0.01,...,CVX240322C00165000,CVX,0.03,None,"Hit point of no confidence, sell.",NaN,0.01,0.03,-0.02,L
3,0.21,2024-03-19T17:06:43.586Z,1.0,0.22,1.0,2024-03-19T17:06:43.608Z,PROD_VAL,PLTR-CDBFC1D-2024-03-19T17-06,2024-03-19T17:06:43.586Z,0.21,...,PLTR240322C00024500,PLTR,0.22,None,"Failed momentum gate, sell.",NaN,0.21,0.22,-0.01,L
4,0.12,2024-03-18T18:06:27.510Z,1.0,0.31,1.0,2024-03-18T18:06:27.535Z,PROD_VAL,UBER-CDBFC1D-2024-03-18T18-06,2024-03-18T18:06:27.510Z,0.12,...,UBER240322C00079000,UBER,0.31,None,"Hit point of no confidence, sell.",NaN,0.12,0.31,-0.19,L
5,0.04,2024-03-18T17:07:13.350Z,1.0,0.09,1.0,2024-03-18T17:07:13.365Z,PROD_VAL,AAPL-CDBFC1D-2024-03-18T17-07,2024-03-18T17:07:13.350Z,0.04,...,AAPL240322C00185000,AAPL,0.09,None,"Hit point of no confidence, sell.",NaN,0.04,0.09,-0.05,L
6,0.05,2024-03-18T19:06:10.015Z,1.0,0.11,1.0,2024-03-18T19:06:10.039Z,PROD_VAL,PYPL-CDBFC1D-2024-03-18T19-06,2024-03-18T19:06:10.015Z,0.05,...,PYPL240322C00069000,PYPL,0.11,None,"Hit point of no confidence, sell.",NaN,0.05,0.11,-0.06,L
7,0.33,2024-03-18T16:06:13.797Z,1.0,0.32,1.0,2024-03-18T16:06:13.821Z,PROD_VAL,UBER-CDBFC1D-2024-03-18T16-06,2024-03-18T16:06:13.797Z,0.33,...,UBER240322C00078000,UBER,0.32,None,"Failed momentum gate, sell.",NaN,0.33,0.32,0.01,W
8,0.13,2024-03-19T16:06:38.162Z,1.0,0.25,1.0,2024-03-19T16:06:38.194Z,PROD_VAL,C-CDBFC1D-2024-03-19T16-06,2024-03-19T16:06:38.162Z,0.13,...,C240322C00060000,C,0.25,None,"Failed momentum gate, sell.",NaN,0.13,0.25,-0.12,L
9,0.15,2024-03-19T14:06:36.715Z,1.0,0.31,1.0,2024-03-19T14:06:36.854Z,PROD_VAL,XOM-CDBFC1D-2024-03-19T14-06,2024-03-19T14:06:36.715Z,0.15,...,XOM240322C00115000,XOM,0.31,None,"Failed momentum gate, sell.",NaN,0.15,0.31,-0.16,L


In [78]:
orders.to_csv('/Users/diz/Documents/Projects/APE-Research/APE-Backtester/backtesting_data/cls/sample_storage/sample_roc_cdbfc1d.csv')